# 13. Testing ways to generate recommendations for improving modules

In [ ]:
import pandas as pd

from google.colab import drive

# Mounting Google Drive
drive.mount('/content/drive', force_remount=True) # Adding force_remount=True to force the remounting process
df_reviews = pd.read_csv("/content/drive/MyDrive/final_processed_primary_secondary_emotion.csv")

# Need to make some kind of priority, e.g. which topic is most important to user

### Idea Behind Teacher Recommendations:

**Main Option 1: Rule-Based**  
- Analyze time-series sentiment and emotion data for each topic over the past 3 months.  
- Identify topics with consistently negative sentiment/emotion for improvement and consistently positive sentiment/emotion for recognition.  
- Use predefined rules and a dictionary of improvement strategies to generate actionable feedback.  
  #### Possibly:
  - Could combine with Reinforcement Learning to recommend strategies that lead to positive outcomes for the teacher - indicates by a teacher 'liking' a recommendation:
  * After receiving the generated recommendation, the teacher can provide feedback on its usefulness (e.g., "helpful," "not helpful").

  * The system learns from the teacher's responses to adjust future feedback generation. Positive feedback (e.g., teacher liked the recommendation) increases the likelihood of similar suggestions being generated in the future, while negative feedback reduces the chances of repeating ineffective recommendations.

  * Reward Mechanism: Use a reward-based system where the more a teacher likes certain suggestions, the more those suggestions are prioritized in future recommendations.

**Main Option 2: LLMs**  
- Pass a summary of each module's sentiment, emotion, and topic trends to an LLM.  
- Prompt the LLM to craft detailed and personalised feedback for improvement and recognition based on the trends.  



In [ ]:
import pandas as pd

# Sample dataset with sentiment labels
data = {
    "name": ["Machine Learning", "Programming for Everybody (Getting Started with Python)",
             "Programming Languages, Part A", "The Data Scientist’s Toolbox",
             "Using Databases with Python", "Using Python to Access Web Data",
             "Introduction to Data Science in Python", "Python Basics",
             "Algorithmic Toolbox", "Information Security: Context and Introduction"],
    "sentiment_from_rating": ["positive", "positive", "neutral", "positive", "negative",
                              "negative", "positive", "positive", "neutral", "positive"],
    "primary_emotion_label": ["joy", "joy", "neutral", "joy", "sadness",
                              "sadness", "joy", "joy", "neutral", "joy"],
    "sentiment_label_from_review": ["positive", "positive", "neutral", "positive", "negative",
                                    "negative", "positive", "positive", "neutral", "positive"],
    "topic_summary": [
        ["Instructor", "Course Material"], ["Learning Outcomes", "Instructor"],
        ["Course Material", "Assignments"], ["Utility and Usefulness"],
        ["Learning Outcomes", "Course Material"], ["Assignments", "Exams"],
        ["Utility and Usefulness"], ["Learning Outcomes", "Course Engagement"],
        ["Course Material"], ["Instructor"]
    ],
    "reviews_date": ["2023-11", "2023-10", "2023-09", "2023-08", "2023-07",
                     "2023-06", "2023-05", "2023-04", "2023-03", "2023-02"],
    "rating": [5.0, 4.5, 3.0, 4.8, 2.5, 2.9, 4.9, 4.6, 3.5, 4.4],
    "emotion_label": ["enjoyment/happiness", "enjoyment/happiness", "neutral",
                      "enjoyment/happiness", "sadness", "sadness",
                      "enjoyment/happiness", "enjoyment/happiness", "neutral",
                      "enjoyment/happiness"]
}

# Create a DataFrame
df = pd.DataFrame(data)

# List of modules to generate recommendations for
modules = [
    'Machine Learning', 'Programming for Everybody (Getting Started with Python)',
    'Programming Languages, Part A', 'The Data Scientist’s Toolbox',
    'Using Databases with Python', 'Using Python to Access Web Data',
    'Introduction to Data Science in Python', 'Python Basics',
    'Algorithmic Toolbox', 'Information Security: Context and Introduction'
]

# Function to generate recommendations with suggestions
def generate_teacher_recommendations_with_suggestions(df, modules):
    recommendations = {}

    # Define potential improvement strategies
    improvement_suggestions = {
        "Instructor": "Consider improving lecture delivery, providing clearer examples, and increasing interactivity during sessions.",
        "Course Material": "Update the material with real-world examples, ensure resources are comprehensive, and include multimedia aids.",
        "Assignments": "Ensure instructions are clear, make them relevant to learning outcomes, and provide detailed feedback.",
        "Learning Outcomes": "Clarify the objectives, align them with course content, and regularly assess if students understand them.",
        "Exams": "Focus on creating fair, clear, and well-structured assessments that reflect the taught material.",
        "Utility and Usefulness": "Highlight the real-world applicability of the course content, including case studies or practical examples.",
        "Course Engagement": "Introduce group activities, Q&A sessions, or discussion forums to encourage active participation."
    }

    for module in modules:
        module_data = df[df['name'] == module]

        if module_data.empty:
            continue

        # Retrieve topics and recent data
        topics = module_data['topic_summary'].iloc[0]  # Assume topic_summary is a list of topics
        time_series = module_data[['reviews_date', 'sentiment_label_from_review',
                                   'primary_emotion_label', 'topic_summary']].copy()

        # Sort data by date for time-series analysis
        time_series['reviews_date'] = pd.to_datetime(time_series['reviews_date'])
        time_series = time_series.sort_values(by='reviews_date', ascending=True)

        actionable_feedback = []

        for topic in topics:
            # Filter for rows where the topic is mentioned
            topic_data = time_series[time_series['topic_summary'].apply(lambda x: topic in x)].copy()

            # Focus on the last 3 months
            last_3_months = topic_data.iloc[-3:] if len(topic_data) >= 3 else topic_data

            # Check sentiment and emotion consistency
            recent_sentiments = last_3_months['sentiment_label_from_review'].value_counts()
            recent_emotions = last_3_months['primary_emotion_label'].value_counts()

            # Poor performance criteria
            if 'negative' in recent_sentiments and recent_sentiments['negative'] == len(last_3_months):
                actionable_feedback.append(
                    f"Focus on improving {topic} as it has received consistently poor sentiment in the last 3 months. "
                    f"Suggestion: {improvement_suggestions.get(topic, 'Consider gathering specific student feedback to address the issue.')}"
                )
            elif 'sadness' in recent_emotions and recent_emotions['sadness'] == len(last_3_months):
                actionable_feedback.append(
                    f"Focus on improving {topic} as it has consistently caused sadness in the last 3 months. "
                    f"Suggestion: {improvement_suggestions.get(topic, 'Consider making the topic more engaging and approachable.')}"
                )

            # Positive performance criteria
            elif 'positive' in recent_sentiments and recent_sentiments['positive'] == len(last_3_months):
                actionable_feedback.append(f"Continue doing excellent work on {topic} as it has received consistently positive sentiment in the last 3 months.")
            elif 'joy' in recent_emotions and recent_emotions['joy'] == len(last_3_months):
                actionable_feedback.append(f"Leverage positive feedback on {topic} as it has consistently evoked joy in the last 3 months.")

        # General Outlook
        general_outlook = f"Over the course of the module '{module}', most reviews are {module_data['sentiment_from_rating'].iloc[0]} with students frequently feeling {module_data['primary_emotion_label'].iloc[0]}."

        # Generate recommendation
        recommendations[module] = {
            "General Outlook": general_outlook,
            "Actionable Feedback": actionable_feedback if actionable_feedback else ["No specific feedback for improvement or recognition."]
        }

    return recommendations

# Generate teacher recommendations
teacher_recommendations = generate_teacher_recommendations_with_suggestions(df, modules)

# Display recommendations
for module, feedback in teacher_recommendations.items():
    print(f"--- Recommendations for Module: {module} ---")
    for key, value in feedback.items():
        if isinstance(value, list):
            print(f"{key}:")
            for action in value:
                print(f"  - {action}")
        else:
            print(f"{key}: {value}")
    print("\n")


--- Recommendations for Module: Machine Learning ---
General Outlook: Over the course of the module 'Machine Learning', most reviews are positive with students frequently feeling joy.
Actionable Feedback:
  - Continue doing excellent work on Instructor as it has received consistently positive sentiment in the last 3 months.
  - Continue doing excellent work on Course Material as it has received consistently positive sentiment in the last 3 months.


--- Recommendations for Module: Programming for Everybody (Getting Started with Python) ---
General Outlook: Over the course of the module 'Programming for Everybody (Getting Started with Python)', most reviews are positive with students frequently feeling joy.
Actionable Feedback:
  - Continue doing excellent work on Learning Outcomes as it has received consistently positive sentiment in the last 3 months.
  - Continue doing excellent work on Instructor as it has received consistently positive sentiment in the last 3 months.


--- Recommen

# Trying to incorporate AI chatbot

In [ ]:
!pip install transformers torch

In [ ]:
import os
import google.generativeai as genai


genai.configure(api_key='SECRET')

# Create the model
generation_config = {
  "temperature": 1,
  "top_p": 0.95,
  "top_k": 40,
  "max_output_tokens": 8192,
  "response_mime_type": "text/plain",
}

model = genai.GenerativeModel(
  model_name="gemini-2.0-flash-exp",
  generation_config=generation_config,
)

chat_session = model.start_chat(
  history=[
    {
      "role": "user",
      "parts": [
        "something to give recommendations to teachers based on student feedback on modules that they teach\n",
      ],
    },
    {
      "role": "model",
      "parts": [
        "Okay, let's craft something that can provide teachers with actionable recommendations based on student feedback on their modules. Here's a breakdown of how it could work, along with examples:\n\n**Core Components:**\n\n1. **Feedback Collection:**\n   * **Structured Surveys:** This is the most common method. Questions could cover:\n      * **Clarity of Content:** (e.g., Was the material presented in a clear and understandable way?)\n      * **Engagement:** (e.g., Did you find the module engaging? Did it hold your interest?)\n      * **Organization:** (e.g., Was the module well-organized? Did the flow of topics make sense?)\n      * **Usefulness:** (e.g., How relevant was the material to your learning goals?)\n      * **Pacing:** (e.g., Was the pace of the module appropriate?)\n      * **Assessment:** (e.g., Were the assessments fair and aligned with the content?)\n      * **Instructor's Role:** (e.g., Was the instructor responsive and helpful?)\n      * **Overall Experience:** (e.g., How satisfied were you with the module overall?)\n   * **Open-Ended Feedback:** Allow students to provide written comments on what they liked, what they didn't, and suggestions for improvement.\n   * **Anonymous Feedback:** Encourage honest responses by ensuring anonymity.\n\n2. **Data Analysis:**\n   * **Quantitative Analysis:**\n      * **Average Scores:** Calculate average scores for each question to identify areas of strength and weakness.\n      * **Frequency Distributions:** Analyze how often students selected specific responses (e.g., how many students rated a question as \"Excellent\").\n      * **Trend Analysis:** Track changes in scores over time to monitor improvement or decline in different areas.\n   * **Qualitative Analysis:**\n      * **Theme Identification:** Look for common themes and patterns in the open-ended feedback.\n      * **Sentiment Analysis:** Use tools to analyze the overall tone (positive, negative, neutral) of the comments.\n\n3. **Recommendation Generation:** This is the key part. Here's how to generate meaningful recommendations:\n   * **Based on Quantitative Data:**\n      * **Low-Scoring Areas:** If a question consistently scores low, suggest specific actions (e.g., \"The question 'Clarity of Content' scored below average. Consider providing more detailed examples or explanations.\")\n      * **Inconsistent Ratings:** If there's a wide spread of ratings, this might indicate that the content appeals to some but not others. Recommend offering optional extensions or different approaches to the material.\n   * **Based on Qualitative Data:**\n       * **Recurring Themes:** If multiple students mention that the pacing is too fast, recommend slowing down. If multiple students mention that the instructor's responsiveness is good, highlight this strength.\n       * **Specific Suggestions:** If a student suggests adding more real-world examples, suggest incorporating them.\n   * **Focus on Actionable Advice:** Recommendations should be specific and easily translated into concrete changes. Avoid vague statements.\n   * **Provide Context:** Present data in a way that is clear and understandable for the teacher, along with comparisons to previous years or other similar modules (if available).\n   * **Highlight Strengths:**  Don't focus solely on weaknesses.  Also include positive feedback to reinforce what is working well.\n\n**Example Scenario and Recommendations:**\n\n**Scenario:**\n\n* **Module:** \"Introduction to Programming\"\n* **Feedback:**\n   * **Quantitative:**  Low average score on the \"Clarity of Content\" question.\n   * **Qualitative:**  Multiple students commented that \"the concepts were confusing\" and \"the examples were too abstract.\"\n\n**Generated Recommendations:**\n\n* **Recommendation 1 (Quantitative):**  \"The average score for 'Clarity of Content' was 3.2 out of 5, which is below the average for similar introductory modules. Consider reviewing the way key programming concepts are presented. Make use of visual aids, analogies, and step-by-step explanations to enhance clarity.\"\n* **Recommendation 2 (Qualitative):** \"Multiple students commented that the concepts were confusing and the examples were too abstract.  Consider incorporating more real-world examples that are directly relatable to student's experiences. For instance, instead of using abstract variables, try a grocery shopping analogy.\"\n* **Recommendation 3 (Qualitative):** \"Students also suggested breaking down the code examples into smaller steps. For the next iteration, try creating code walkthroughs where you explain what each line does and why.\"\n* **Positive Reinforcement:** \"Students consistently praised your enthusiasm and passion for the subject.\"\n* **Next Steps:** \"We recommend reviewing the module materials and incorporating these suggestions for the next iteration. A follow-up session in a few weeks can further help you address any concerns.\"\n\n**Key Features of a Good Recommendation System:**\n\n* **User-Friendly Interface:** The tool should be easy for teachers to use and understand, even if they are not data experts.\n* **Customizable:** Allow teachers to focus on specific aspects of their teaching.\n* **Regular Updates:** The tool should provide updated data on a regular basis, allowing teachers to track their progress.\n* **Support and Resources:** Provide resources and training to help teachers understand the recommendations and implement changes.\n\n**Technology to Use:**\n\n* **Survey Platforms:** (e.g., Google Forms, SurveyMonkey, Typeform)\n* **Data Analysis Tools:** (e.g., Excel, Google Sheets, Python with Pandas, R)\n* **Visualization Tools:** (e.g., Tableau, Power BI)\n* **Natural Language Processing (NLP):**  For more advanced qualitative analysis\n\nBy implementing a comprehensive system for collecting, analyzing, and generating recommendations based on student feedback, teachers can gain valuable insights into the effectiveness of their modules, leading to a more engaging and effective learning experience for students. Remember that this is an iterative process. Regularly collect feedback and refine your approach to continually improve your teaching.\n",
      ],
    },
  ]
)

response = chat_session.send_message("Given the negative sentiment and expressions of confusion related to the 'Content' topic in module Cybersecurity, suggest three concrete strategies to improve content clarity for the next cycle.")

print(response.text)

Okay, let's address the negative sentiment and confusion surrounding the 'Content' topic within the Cybersecurity module. Here are three concrete strategies to improve content clarity for the next cycle, focusing on practical, actionable changes:

**1. Deconstruct and Modularize Complex Concepts with Visual Aids:**

* **Problem:** Cybersecurity concepts are often inherently complex and interconnected, making it hard for students to grasp the big picture or individual components. Students may feel overwhelmed by jargon and abstract ideas.
* **Strategy:**
    * **Break Down into Smaller Modules:** Instead of presenting a large block of information, break down complex topics into smaller, digestible modules. Each module should focus on a single key concept or skill.
    * **Create Visual Flowcharts and Diagrams:** Use visual aids like flowcharts, diagrams, and infographics to illustrate the relationships between different concepts, processes, and network components. For example, instead o

given that I want to provide a teacher with recommendations to improve their module based on student feedback, what data results can I base this on? I have data on: rating (1-5), sentiment(positive/negative), emotion(joy, surprise, admiration etc.), the date of the review, the topic(s) of the review (exam, grading, learning outcomes etc.) and what I want to do is provide an overall outlook/analysis for each module and use that as a prompt to build the teacher recommendations to improve their modules

Ideal Prompt:
The student feedback for the module "Advanced Physics" includes the following insights:
- Average Rating: 4.1
- Sentiment: 65% positive, 35% negative
- Key Emotions: Joy (40%), Admiration (25%), Frustration (20%)
- Feedback by Topic:
  - Exams: Concerns about clarity and time constraints.
  - Grading: Perceived subjectivity in grading.
  - Learning Outcomes: Appreciation for conceptual understanding but limited practical application.
  
Time-Based Trend: Negative feedback peaked during midterm assessments.

Based on these insights, generate actionable recommendations for the teacher to improve the module.


# To build an effective prompt, I need to add subtopics for each topic
How to do this?
Already have primary labels.
Need to add secondary labels based on primary labels.
E.g., many reviews could be about exam, but one could be about the difficulty of the exam, whereas another could be about the time constraints in the exam.
How am I going to do this for such a large volume of data though? > 30k reviews!!

In [ ]:
import pandas as pd
import numpy as np

# Define subtopic keywords for each topic
subtopic_keywords = {
    "Exam": {
        "difficulty": ["hard", "challenging", "difficult"],
        "fairness": ["fair", "unfair", "bias"],
        "time constraints": ["time", "duration", "too short", "too long"],
        "clarity": ["unclear", "confusing", "instructions"]
    },
    "Assignment": {
        "help": ["help", "support"],
        "resources": ["material", "resources", "readings"],
        "complexity": ["complex", "difficult", "challenging"],
        "deadline": ["deadline", "extension", "due date"]
    },
    "Lecturer": {
        "engagement": ["engaging", "interactive", "boring"],
        "style": ["style", "approach", "teaching"],
        "availability": ["available", "unavailable", "office hours"],
        "clarity": ["clear", "unclear", "explain"]
    },
    "Grading": {
        "fairness": ["fair", "unfair", "bias"],
        "transparency": ["transparent", "unclear"],
        "feedback": ["feedback", "comments", "detailed"]
    }
}

def detect_subtopic(text, topic):
    if pd.isna(text):
        return "General"

    subtopics = subtopic_keywords.get(topic, {})
    for subtopic, keywords in subtopics.items():
        if any(keyword in text.lower() for keyword in keywords):
            return subtopic
    return "General"

def detect_subtopic(text, topic):
    if pd.isna(text):
        return "General"

    # Normalize text for case-insensitive matching
    text_lower = text.lower()
    subtopics = subtopic_keywords.get(topic, {})
    detected_subtopics = []

    for subtopic, keywords in subtopics.items():
        # Check if any keyword is present in the text
        if any(keyword in text_lower for keyword in keywords):
            detected_subtopics.append(subtopic)

    return ", ".join(detected_subtopics) if detected_subtopics else "General"

def analyze_feedback(df):
    insights = {}

    topics = df["topic"].unique()

    # Initialize general analysis metrics
    total_feedback = len(df)
    avg_rating = df["rating"].mean()
    sentiment_dist = df["sentiment"].value_counts(normalize=True) * 100
    emotion_dist = df["emotion"].value_counts(normalize=True) * 100

    feedback_summary = {
        "average_rating": avg_rating,
        "sentiment_distribution": {sentiment: f"{percentage:.1f}%" for sentiment, percentage in sentiment_dist.items()},
        "key_emotions": {emotion: f"{percentage:.1f}%" for emotion, percentage in emotion_dist.items()},
        "topic_feedback": {}
    }

    # Time-based trend if available
    if "date" in df.columns:
        df["date"] = pd.to_datetime(df["date"])
        trend_over_time = df.groupby(df["date"].dt.to_period("M"))["rating"].mean()
        if not trend_over_time.empty:
            feedback_summary["time_based_trend"] = trend_over_time.idxmin().strftime("%B %Y") + " saw the lowest ratings."
        else:
            feedback_summary["time_based_trend"] = "No significant time-based trends detected."

    # Analyze by topic
    for topic in topics:
        topic_data = df[df["topic"] == topic]
        subtopic_data = topic_data.copy()

        # Detect subtopics
        subtopic_data["subtopic"] = subtopic_data["review_text"].apply(lambda x: detect_subtopic(x, topic))
        subtopic_counts = subtopic_data["subtopic"].value_counts().to_dict()

        # Highlight key concerns by subtopic
        key_concerns = [f"{subtopic}" for subtopic, count in subtopic_counts.items() if count > 1]

        # Add insights for each topic
        feedback_summary["topic_feedback"][topic] = f"Key concerns about {', '.join(key_concerns) if key_concerns else 'no major issues detected'}."

    return feedback_summary

def format_feedback_summary(summary):
    """
    Format the feedback summary into a readable string.
    """
    result = []

    result.append(f"**Average Rating:** {summary['average_rating']:.1f}")

    # Sentiment Analysis
    sentiment_details = ", ".join([f"{sentiment}: {percentage}" for sentiment, percentage in summary["sentiment_distribution"].items()])
    result.append(f"**Sentiment:** {sentiment_details}")

    # Key Emotions
    emotion_details = ", ".join([f"{emotion}: {percentage}" for emotion, percentage in summary["key_emotions"].items()])
    result.append(f"**Key Emotions:** {emotion_details}")

    # Feedback by Topic
    result.append("**Feedback by Topic:**")
    for topic, feedback in summary["topic_feedback"].items():
        result.append(f"- {topic}: {feedback}")

    # Time-based trends
    result.append(f"**Time-Based Trend:** {summary['time_based_trend']}")

    return "\n".join(result)

# Example DataFrame
data = {
    "topic": ["Exam", "Exam", "Assignment", "Lecturer", "Grading", "Exam", "Assignment"],
    "rating": [3, 2, 2, 5, 2, 3, 1],
    "sentiment": ["negative", "negative", "negative", "positive", "negative", "negative", "negative"],
    "emotion": ["frustration", "frustration", "frustration", "joy", "frustration", "frustration", "frustration"],
    "date": ["2025-01-01", "2025-01-15", "2025-01-10", "2025-01-12", "2025-01-20", "2025-02-01", "2025-01-25"],
    "review_text": [
        "The exam was too hard.",
        "Exam timing was unfair.",
        "The assignment was confusing and difficult.",
        "The lecturer was engaging and clear.",
        "The grading felt unfair and lacked transparency.",
        "Unclear exam instructions.",
        "Assignment deadlines were unreasonable."
    ]
}

df = pd.DataFrame(data)

# Analyze the feedback and format the results
insights = analyze_feedback(df)
formatted_feedback = format_feedback_summary(insights)

print(formatted_feedback)


**Average Rating:** 2.6
**Sentiment:** negative: 85.7%, positive: 14.3%
**Key Emotions:** frustration: 85.7%, joy: 14.3%
**Feedback by Topic:**
- Exam: Key concerns about no major issues detected.
- Assignment: Key concerns about no major issues detected.
- Lecturer: Key concerns about no major issues detected.
- Grading: Key concerns about no major issues detected.
**Time-Based Trend:** January 2025 saw the lowest ratings.


In [ ]:
import pandas as pd

# Sample data setup as described
data = {
    "review_text": [
        "The exam was too difficult and the time was not enough.",
        "Assignments were fair but lacked proper resource guidance.",
        "The lecturer's teaching style is boring and unclear.",
        "The grading felt unfair and lacked transparency."
    ],
    "topic": ["Exam", "Assignment", "Lecturer", "Grading"],
    "rating": [2, 3, 1, 2],
    "subtopic": ["Time Constraint, Difficulty", "Resources", "Teaching Style", "Fairness"],
    "sentiment": ["negative", "negative", "negative", "negative"],
    "emotion": ["frustration", "frustration", "frustration", "frustration"],
    "date": ["2025-01-15", "2025-01-20", "2025-01-18", "2025-01-25"]
}

# Create DataFrame
df = pd.DataFrame(data)

# **1. Calculate Average Rating**
average_rating = df["rating"].mean()

# **2. Sentiment Distribution**
sentiment_counts = df["sentiment"].value_counts(normalize=True) * 100
sentiment_distribution = {
    "positive": sentiment_counts.get("positive", 0),
    "negative": sentiment_counts.get("negative", 0)
}

# **3. Key Emotions**
emotion_counts = df["emotion"].value_counts(normalize=True) * 100
emotion_distribution = emotion_counts.to_dict()

# **4. Feedback by Topic**
topic_feedback = {}
for topic in df["topic"].unique():
    # Filter reviews by topic
    topic_reviews = df[df["topic"] == topic]

    # Calculate negative sentiment percentage for the current topic
    negative_sentiment_percentage = (topic_reviews["sentiment"] == "negative").mean() * 100

    if negative_sentiment_percentage > 50:
        # If more than 50% of reviews are negative, mention the concerns
        subtopics = ', '.join(topic_reviews["subtopic"].unique())
        feedback = f"Main concerns are about {subtopics} of the {topic}."
    else:
        # If less than or equal to 50% of reviews are negative, mention positive feedback
        subtopics = ', '.join(topic_reviews["subtopic"].unique())
        feedback = f"{subtopics} of the {topic} is not concerning by students and is positive."

    topic_feedback[topic] = feedback

# **5. Time-Based Trend**
# Convert date to datetime format and get month-year for comparison
df["date"] = pd.to_datetime(df["date"])
df["month_year"] = df["date"].dt.to_period("M")

# Find the month with the lowest average rating
monthly_avg_rating = df.groupby("month_year")["rating"].mean()
lowest_rating_month = monthly_avg_rating.idxmin()

# **Create Output**
output = f"""
**Average Rating:** {average_rating:.1f}
**Sentiment:** negative: {sentiment_distribution['negative']:.1f}%, positive: {sentiment_distribution['positive']:.1f}%
**Key Emotions:** {', '.join([f'{emotion}: {value:.1f}%' for emotion, value in emotion_distribution.items()])}
**Feedback by Topic:**
"""

for topic, feedback in topic_feedback.items():
    output += f"- {topic}: {feedback}\n"

output += f"**Time-Based Trend:** {lowest_rating_month} saw the lowest ratings."

# Print the output
print(output)



**Average Rating:** 2.0
**Sentiment:** negative: 100.0%, positive: 0.0%
**Key Emotions:** frustration: 100.0%
**Feedback by Topic:**
- Exam: Main concerns are about Time Constraint, Difficulty of the Exam.
- Assignment: Main concerns are about Resources of the Assignment.
- Lecturer: Main concerns are about Teaching Style of the Lecturer.
- Grading: Main concerns are about Fairness of the Grading.
**Time-Based Trend:** 2025-01 saw the lowest ratings.


Testing the ideal Outlook Summary to use to prompt the LLM

In [ ]:
import os
import google.generativeai as genai


genai.configure(api_key='SECRET')

# Create the model
generation_config = {
  "temperature": 0.7, # need to check temp param
  "top_p": 0.95,
  "top_k": 40,
  "max_output_tokens": 8192,
  "response_mime_type": "text/plain",
}

model = genai.GenerativeModel(
  model_name="gemini-2.0-flash-exp",
  generation_config=generation_config,
)

chat_session = model.start_chat(
  history=[
    {
      "role": "user",
      "parts": [
        "something to give recommendations to teachers to improve modules based on student feedback on modules that they teach\n",
      ],
    },
    {
      "role": "model",
      "parts": [
        "Okay, let's craft something that can provide teachers with actionable recommendations based on student feedback on their modules. Here's a breakdown of how it could work, along with examples:\n\n**Core Components:**\n\n1. **Feedback Collection:**\n   * **Structured Surveys:** This is the most common method. Questions could cover:\n      * **Clarity of Content:** (e.g., Was the material presented in a clear and understandable way?)\n      * **Engagement:** (e.g., Did you find the module engaging? Did it hold your interest?)\n      * **Organization:** (e.g., Was the module well-organized? Did the flow of topics make sense?)\n      * **Usefulness:** (e.g., How relevant was the material to your learning goals?)\n      * **Pacing:** (e.g., Was the pace of the module appropriate?)\n      * **Assessment:** (e.g., Were the assessments fair and aligned with the content?)\n      * **Instructor's Role:** (e.g., Was the instructor responsive and helpful?)\n      * **Overall Experience:** (e.g., How satisfied were you with the module overall?)\n   * **Open-Ended Feedback:** Allow students to provide written comments on what they liked, what they didn't, and suggestions for improvement.\n   * **Anonymous Feedback:** Encourage honest responses by ensuring anonymity.\n\n2. **Data Analysis:**\n   * **Quantitative Analysis:**\n      * **Average Scores:** Calculate average scores for each question to identify areas of strength and weakness.\n      * **Frequency Distributions:** Analyze how often students selected specific responses (e.g., how many students rated a question as \"Excellent\").\n      * **Trend Analysis:** Track changes in scores over time to monitor improvement or decline in different areas.\n   * **Qualitative Analysis:**\n      * **Theme Identification:** Look for common themes and patterns in the open-ended feedback.\n      * **Sentiment Analysis:** Use tools to analyze the overall tone (positive, negative, neutral) of the comments.\n\n3. **Recommendation Generation:** This is the key part. Here's how to generate meaningful recommendations:\n   * **Based on Quantitative Data:**\n      * **Low-Scoring Areas:** If a question consistently scores low, suggest specific actions (e.g., \"The question 'Clarity of Content' scored below average. Consider providing more detailed examples or explanations.\")\n      * **Inconsistent Ratings:** If there's a wide spread of ratings, this might indicate that the content appeals to some but not others. Recommend offering optional extensions or different approaches to the material.\n   * **Based on Qualitative Data:**\n       * **Recurring Themes:** If multiple students mention that the pacing is too fast, recommend slowing down. If multiple students mention that the instructor's responsiveness is good, highlight this strength.\n       * **Specific Suggestions:** If a student suggests adding more real-world examples, suggest incorporating them.\n   * **Focus on Actionable Advice:** Recommendations should be specific and easily translated into concrete changes. Avoid vague statements.\n   * **Provide Context:** Present data in a way that is clear and understandable for the teacher, along with comparisons to previous years or other similar modules (if available).\n   * **Highlight Strengths:**  Don't focus solely on weaknesses.  Also include positive feedback to reinforce what is working well.\n\n**Example Scenario and Recommendations:**\n\n**Scenario:**\n\n* **Module:** \"Introduction to Programming\"\n* **Feedback:**\n   * **Quantitative:**  Low average score on the \"Clarity of Content\" question.\n   * **Qualitative:**  Multiple students commented that \"the concepts were confusing\" and \"the examples were too abstract.\"\n\n**Generated Recommendations:**\n\n* **Recommendation 1 (Quantitative):**  \"The average score for 'Clarity of Content' was 3.2 out of 5, which is below the average for similar introductory modules. Consider reviewing the way key programming concepts are presented. Make use of visual aids, analogies, and step-by-step explanations to enhance clarity.\"\n* **Recommendation 2 (Qualitative):** \"Multiple students commented that the concepts were confusing and the examples were too abstract.  Consider incorporating more real-world examples that are directly relatable to student's experiences. For instance, instead of using abstract variables, try a grocery shopping analogy.\"\n* **Recommendation 3 (Qualitative):** \"Students also suggested breaking down the code examples into smaller steps. For the next iteration, try creating code walkthroughs where you explain what each line does and why.\"\n* **Positive Reinforcement:** \"Students consistently praised your enthusiasm and passion for the subject.\"\n* **Next Steps:** \"We recommend reviewing the module materials and incorporating these suggestions for the next iteration. A follow-up session in a few weeks can further help you address any concerns.\"\n\n**Key Features of a Good Recommendation System:**\n\n* **User-Friendly Interface:** The tool should be easy for teachers to use and understand, even if they are not data experts.\n* **Customizable:** Allow teachers to focus on specific aspects of their teaching.\n* **Regular Updates:** The tool should provide updated data on a regular basis, allowing teachers to track their progress.\n* **Support and Resources:** Provide resources and training to help teachers understand the recommendations and implement changes.\n\n**Technology to Use:**\n\n* **Survey Platforms:** (e.g., Google Forms, SurveyMonkey, Typeform)\n* **Data Analysis Tools:** (e.g., Excel, Google Sheets, Python with Pandas, R)\n* **Visualization Tools:** (e.g., Tableau, Power BI)\n* **Natural Language Processing (NLP):**  For more advanced qualitative analysis\n\nBy implementing a comprehensive system for collecting, analyzing, and generating recommendations based on student feedback, teachers can gain valuable insights into the effectiveness of their modules, leading to a more engaging and effective learning experience for students. Remember that this is an iterative process. Regularly collect feedback and refine your approach to continually improve your teaching.\n",
      ],
    },
  ]
)

response = chat_session.send_message("Given the following summary of data for this module called 'Programming' : Average Rating:** 2.6 **Sentiment:** negative: 85.7%, positive: 14.3% **Key Emotions:** frustration: 85.7%, joy: 14.3% **Feedback by Topic:** - Exam: Key concerns about no major issues detected. - Assignment: Key concerns about no major issues detected. - Lecturer: Key concerns about no major issues detected. - Grading: Key concerns about no major issues detected. **Time-Based Trend:** January 2025 saw the lowest ratings., suggest three concrete strategies to improve content clarity for the next cycle.")

print(response.text)

Okay, this is a very clear and concerning picture for the "Programming" module. The low average rating (2.6), overwhelmingly negative sentiment (85.7%), high levels of frustration (85.7%), and lack of specific concerns in the feedback topics all point towards a fundamental issue with the module's delivery, likely related to clarity and perhaps engagement. The January 2025 dip is also a crucial piece of information, suggesting a potential change or issue introduced during that period.

Here are three concrete strategies to improve content clarity for the next cycle, specifically addressing the identified problems:

**Strategy 1: "Deconstruct & Rebuild" Core Concepts with Micro-Learning Modules**

*   **Problem Addressed:** The overall lack of clarity and high frustration suggests students are struggling with the fundamental concepts. The lack of specific feedback on topics suggests the problem is pervasive across the module, not isolated.
*   **Actionable Steps:**
    1.  **Identify Cor

In [ ]:
import os
import google.generativeai as genai

genai.configure(api_key='SECRET')

# Create the model
generation_config = {
  "temperature": 1,
  "top_p": 0.95,
  "top_k": 40,
  "max_output_tokens": 8192,
  "response_mime_type": "text/plain",
}

model = genai.GenerativeModel(
  model_name="gemini-2.0-flash-exp",
  generation_config=generation_config,
)

chat_session = model.start_chat(
  history=[
    {
      "role": "user",
      "parts": [
        "I want to prompt the chatbot using data like this: \n**Average Rating:** 2.0\n**Sentiment:** negative: 100.0%, positive: 0.0%\n**Key Emotions:** frustration: 100.0%\n**Feedback by Topic:**\n- Exam: Main concerns are about Time Constraint, Difficulty of the Exam.\n- Assignment: Main concerns are about Resources of the Assignment.\n- Lecturer: Main concerns are about Teaching Style of the Lecturer.\n- Grading: Main concerns are about Fairness of the Grading.\n**Time-Based Trend:** 2025-01 saw the lowest ratings. To ask if for recommendations to improve a module if the feedback is bad, or give praise. for example, feedback like, to address the student concerns about time constraints, you may want to allow more time for the exam or reduce the number of questions",
      ],
    },
    {
      "role": "model",
      "parts": [
        "Okay, I understand. You want me to use this structured feedback data to generate recommendations (if the feedback is bad) or praise (if the feedback is good) aimed at improving a module or reinforcing positive aspects. \n\nHere's how I can process that data and what kind of prompts I'll generate based on the provided information:\n\n**General Approach:**\n\n1.  **Identify Overall Sentiment:** The first step is to look at the `Average Rating` and `Sentiment` to determine the overall tone of the feedback. In your example, it's clearly negative due to the low rating and 100% negative sentiment.\n2.  **Extract Key Concerns:**  I will then extract the main concerns from the `Key Emotions` and `Feedback by Topic` sections.\n3.  **Time Trend Analysis:** Use `Time-Based Trend` to add more context and see if the issues are recent or persistent.\n4.  **Generate Specific Recommendations/Praise:** I will formulate actionable suggestions tailored to the identified issues, and if the feedback were positive, I would provide praise related to the specific strengths.\n\n**Example Based on YOUR Provided Data:**\n\nHere's how I would process the data you gave me and what prompt I would generate:\n\n**Analysis:**\n\n*   **Overall:** Negative feedback (2.0 average rating, 100% negative sentiment).\n*   **Key Emotions:** High frustration.\n*   **Specific Concerns:**\n    *   **Exam:** Time constraints and difficulty.\n    *   **Assignment:** Resources inadequacy.\n    *   **Lecturer:** Teaching style.\n    *   **Grading:** Fairness.\n*   **Time Trend:** Issues were worst in 2025-01.\n\n**Generated Prompt:**\n\n\"The module received overwhelmingly negative feedback, with an average rating of 2.0 and 100% negative sentiment. Students express high levels of frustration. Specifically, they have significant concerns about:\n\n*   **Exam:** The exam is perceived as too time-constrained and too difficult. To address this, consider allowing more time for the exam or reducing the number of questions. You may also want to review the difficulty level to ensure it is appropriate for the learning outcomes.\n*   **Assignment:** Students are dissatisfied with the resources provided for the assignments. Improving the clarity, accessibility and relevance of the resources is critical.\n*  **Lecturer:**  Students have issues with the lecturer's teaching style. To address this, please review and enhance the teaching approach with more active, engaging methods.\n*   **Grading:** Students believe the grading is not fair.  You should re-evaluate the grading criteria and ensure they are transparent, consistent, and clearly communicated to students.\n\nThe issues were particularly pronounced in 2025-01. It is imperative to address these concerns urgently to enhance student satisfaction and learning outcomes in this module.\"\n\n**How the Prompt Adapts:**\n\n*   **Positive Feedback:** If the average rating and sentiment were positive, the prompt would shift to highlighting the positive points, for example:\n    *   \"This module received positive feedback with an average rating of 4.5 and 90% positive sentiment. Students are pleased with the...\" \n    *   \"Students praise the lecturer's engagement and accessibility. Please continue the excellent work\"\n*   **Mixed Feedback:** If there was a mix of positive and negative feedback, I would generate prompts that acknowledges both and highlights areas for improvement.\n    *   \"The module received mixed feedback, with an average rating of 3.5. While students expressed satisfaction with the teaching style of the lecturer, there are concerns about the resources provided for assignments.\"\n\n**Key Improvements of the Prompt:**\n\n*   **Actionable Recommendations:**  The prompt doesn't just state problems; it gives concrete, actionable suggestions for improvement.\n*   **Specific and Tailored:** Recommendations are tied directly to specific feedback topics.\n*   **Contextualized:** Time-based trends add context to the issues.\n*   **Urgency (if needed):** If feedback is severely negative, it conveys a sense of urgency.\n\n**How to Use This:**\n\n1.  **Feed the Data:** Give me the structured data like you provided.\n2.  **Receive a Prompt:** I will generate a tailored prompt based on the analysis.\n3.  **Use the Prompt:** Use my prompt as a basis for developing reports, informing instructors, or designing improvements to your educational module.\n\nLet me know if you have any other scenarios you'd like to explore or if you want to refine this approach further! I'm ready to help.\n",
      ],
    },
    {
      "role": "user",
      "parts": [
        "based on this, I would prompt the chatbot like  for example, I am a teacher of a module Programming, and this is the summary of my feedback from my students: Overall: Negative feedback (2.0 average rating, 100% negative sentiment).\n\nKey Emotions: High frustration.\n\nSpecific Concerns:\n\nExam: Time constraints and difficulty.\n\nAssignment: Resources inadequacy.\n\nLecturer: Teaching style.\n\nGrading: Fairness.\n\nTime Trend: Issues were worst in 2025-01. How can I address each concern (i.e,) improve and prevent it to improve my feedback and how do I address the emotions and sentiment",
      ],
    },
    {
      "role": "model",
      "parts": [
        "Okay, I understand. You want to use the summarized feedback to prompt the chatbot to generate actionable strategies for you, as the teacher of the \"Programming\" module, to address the specific concerns, improve future feedback, and manage the negative emotions and sentiment.\n\nHere's how I'd approach prompting the chatbot, along with explanations of why I'm including each element:\n\n**Prompt:**\n\n\"I am a teacher for the 'Programming' module. I recently received student feedback, which indicates significant areas for improvement. Here's a summary of the feedback:\n\n**Overall:** Negative feedback (2.0 average rating, 100% negative sentiment).\n\n**Key Emotions:** High frustration.\n\n**Specific Concerns:**\n\n*   **Exam:** Time constraints and difficulty.\n*   **Assignment:** Resources inadequacy.\n*   **Lecturer:** Teaching style.\n*   **Grading:** Fairness.\n\n**Time Trend:** Issues were worst in 2025-01.\n\nGiven this feedback:\n\n1.  **Actionable Strategies:** Please provide specific, actionable strategies I can implement to address each of the identified concerns (exam, assignment, lecturer, and grading) to improve student learning outcomes and future feedback.\n2.  **Prevention:** How can I proactively address and prevent these issues from recurring in future iterations of the module? Please propose specific steps and methods.\n3.  **Emotional Management:** How can I address the high levels of frustration students are expressing, and improve the overall negative sentiment toward the module? What specific communication or support strategies should I consider?\n4. **Prioritisation:** Based on the above, which areas should I focus on first, and what is the rational for the suggested priortisation?\n\nI'm looking for practical, concrete advice that I can immediately start implementing. Please provide a comprehensive and structured response covering these points.\"\n\n**Explanation of the Prompt Components:**\n\n1.  **Context and Role:**\n    *   \"I am a teacher for the 'Programming' module.\" - This sets the context and tells the chatbot your role and which specific module the feedback is about. This is crucial to get tailored advice.\n2.  **Clear Summary of Feedback:**\n    *   The summary you provided is clearly presented and ensures the chatbot has a good understanding of the problem.\n3.  **Specific Requests:**\n    *   **Actionable Strategies:** Asking for specific, actionable strategies gets you past generic advice and into concrete steps.\n    *   **Prevention:** This forces the chatbot to think proactively and beyond just fixing the current problems.\n    *   **Emotional Management:** This tackles the emotional impact of the poor feedback, which is critical for student well-being and future engagement.\n    *   **Prioritisation:** Helps you to decide where to focus your efforts, given limited time and resources.\n4.  **Emphasis on Practicality:**\n    *   \"I'm looking for practical, concrete advice that I can immediately start implementing.\" - This reinforces that you need real-world advice, not abstract concepts.\n5.  **Request for Structure:**\n    *   \"Please provide a comprehensive and structured response covering these points.\" - This encourages the chatbot to organize its answer for clarity.\n\n**Expected Chatbot Response:**\n\nThe chatbot should now provide you with detailed suggestions that might include:\n\n*   **Exam:**\n    *   **Time Management:** Strategies for better exam time allocation, perhaps practice exams or providing time recommendations.\n    *   **Difficulty:** Suggestions for revising exam questions and how to test material appropriately.\n*   **Assignment:**\n    *   **Resource Improvement:** How to create or find better resources, and how to make them available and understandable.\n    *   **Assignment Clarification:** How to make the assignment goals clearer and more defined.\n*   **Lecturer:**\n    *   **Teaching Style:** Strategies for improving engagement, using different methods, using formative assessments, making more time for individual questions.\n    *   **Communication:** How to have more effective office hours, and respond to queries in a timely manner.\n*   **Grading:**\n    *   **Clarity:** Making grading criteria more transparent and consistent.\n    *   **Fairness:** How to eliminate any perceived bias, ensuring each student is graded fairly.\n*   **Prevention:** Suggestions such as regular surveys, gathering of feedback and changes to the learning methodology.\n*   **Emotional Management:** Open lines of communication, encourage feedback, empathy and acknowledging student struggles.\n*   **Prioritization:** Recommendations on which areas should take precedence and how to prioritize them.\n\n**Why This Prompt Is Effective:**\n\n*   **Comprehensive:** It addresses the problem from all angles.\n*   **Action-Oriented:** It asks for specific, doable steps.\n*   **Empathy Aware:** It addresses both the practical and emotional aspects of the situation.\n*   **Future Focused:** It seeks ways to prevent the problems from recurring.\n\nBy using this prompt, you'll likely receive a comprehensive guide for improving your \"Programming\" module, student satisfaction, and your own teaching approach. Remember to follow up on the initial chatbot response, ask clarifying questions and use this as the start of the solution.\n",
      ],
    },
    {
      "role": "user",
      "parts": [
        "An example of feedback from the chatbot might be \"overall, the feedback on your module is {sentiment}, with students feeling {emotion}. This is mainly because students are concerned about the time constraints and difficulty of the exam, the lack of resources available for Assignments and the teaching style of the lecturer. To address the concerns about exams, consider reducing the number of questions in the paper or extending the time ... something similar for each topic.\" Remember that it is a recommendation/suggestion, don't make it seem like you are an expert and be mindful of the teacher's feelings",
      ],
    },
    {
      "role": "model",
      "parts": [
        "Okay, I understand. You want the chatbot to generate feedback recommendations that are:\n\n*   **Non-authoritative:** Not coming across as an expert, but rather as a helpful assistant offering suggestions.\n*   **Empathetic:** Mindful of the teacher's feelings, especially considering the negative feedback received.\n*   **Specific:** Providing concrete examples, just as mentioned in your example about the exam.\n*   **Structured:** Addressing each topic (exam, assignment, lecturer, grading) individually.\n\nHere's how I'd frame a chatbot response that meets those criteria:\n\n**Example Chatbot Response:**\n\n\"Okay, let's look at this feedback for the 'Programming' module together. It sounds like you've got some areas where students are feeling frustrated, and that can be tough to hear.\n\nOverall, the feedback on your module is **largely negative**, with students feeling **high levels of frustration**. This seems mainly because students are concerned about:\n\n*   **Exam:** The time constraints and difficulty of the exam are a major point of concern. To address this, you might consider reducing the number of questions on the paper or extending the time allowed for the exam. This could help ease the pressure students feel. You could also review the exam questions to ensure they are aligned with the module learning outcomes and difficulty level.\n*   **Assignment:** Students have indicated there is a lack of resources available for assignments. Perhaps reviewing the current resources and seeing if additional material is needed, or if the available material can be made clearer would be helpful. You could consider adding supplemental material, pointers, or more detailed instructions to enhance student understanding.\n*   **Lecturer:** The teaching style of the lecturer is also a concern for students. It could be helpful to explore how the course is being taught to see if there are other teaching approaches that would engage students more effectively. Maybe try to incorporate some more active learning techniques, or look into other methods that could make the content more accessible.\n*   **Grading:** Finally, there are concerns about the fairness of the grading. Reviewing and clarifying the grading criteria with students, and providing rubrics of how grades will be awarded could be a step to consider. This could improve the level of trust and transparency of the grading.\n\nLooking at the time trend, it appears the issues were most pronounced in 2025-01, so this might be a good area to focus on.\n\nThese are just a few ideas, of course. I'm here to help you explore all the options and improve the experience for your students, and I am confident that by addressing these specific points, the situation can improve.\"\n\n**Key Features of This Response:**\n\n*   **Empathetic Opening:** Phrases like \"It sounds like you've got some areas where students are feeling frustrated, and that can be tough to hear\" acknowledges the emotional impact of negative feedback.\n*   **Non-Authoritative Language:** The use of phrases like \"you might consider,\" \"perhaps,\" \"it could be helpful,\" and \"maybe try to\" makes the suggestions feel less like dictates and more like helpful ideas. It takes away the \"expert\" tone and offers advice, not commands.\n*   **Specific Examples:** Each suggestion includes a concrete example of how the identified problem could be addressed. This includes actionable ideas like \"reducing the number of questions\" or \"providing rubrics\".\n*   **Structured Format:** Each concern (exam, assignment, lecturer, grading) is addressed individually for clarity.\n*   **Focus on Improvement:** The response focuses on helping the teacher improve, not dwelling on the negative feedback itself.\n*   **Positive Ending:** The response ends on a note of support and confidence in the teacher's ability to improve the situation.\n*   **Focus on the Data:** The inclusion of \"looking at the time trend, it appears the issues were most pronounced in 2025-01\" reinforces the idea that the suggestions are backed by the data, not arbitrary recommendations.\n\n**How to Use This:**\n\nWhen prompting the chatbot, you can include instructions like:\n\n\"Please generate a response that acknowledges that feedback can be difficult to receive. Your response should be non-authoritative, offering suggestions rather than commands, and mindful of the teacher's feelings. Please provide specific and actionable recommendations. Include a positive ending.\"\n\nBy providing these guidelines to the chatbot, you should receive a response that is more helpful and less likely to overwhelm the teacher with seemingly harsh or prescriptive advice. Remember to further question the chatbot to tailor the advice even further.\n",
      ],
    },
  ]
)

response = chat_session.send_message('''
I am a teacher of a module Programming, and this is the summary of my feedback from my students:
Overall: Negative feedback (2.0 average rating, 100% negative sentiment).

Key Emotions: High frustration.

Specific Concerns:

Exam: Time constraints and difficulty.

Assignment: Resources inadequacy.

Lecturer: Teaching style.

Grading: Fairness.

Time Trend: Issues were worst in 2025-01. How can I address each concern (i.e,) improve and prevent it to improve my feedback and how do I address the emotions and sentiment
''')

print(response.text)

Okay, I understand. You're a teacher of a "Programming" module, and you've received some challenging feedback. You want to know how to address each concern, prevent future issues, and manage the negative emotions and sentiment expressed by your students.

Here's a breakdown of how we can approach this, followed by a structured response that you can use (or adapt) as a prompt for a chatbot:

**Understanding the Issues:**

*   **Overall Negative Feedback:** A 2.0 average rating and 100% negative sentiment indicate a significant problem with the module's current state.
*   **High Frustration:** This suggests students are not just dissatisfied but also actively struggling with the course.
*   **Specific Concerns:**
    *   **Exam:** Time constraints and difficulty are both significant issues, implying either a poorly designed exam or a gap between student preparation and exam expectations.
    *   **Assignment:** Inadequate resources signal that students lack the necessary support to compl

In [ ]:
import os
import google.generativeai as genai

genai.configure(api_key='SECRET')

# Create the model
generation_config = {
  "temperature": 1,
  "top_p": 0.95,
  "top_k": 40,
  "max_output_tokens": 8192,
  "response_mime_type": "text/plain",
}

model = genai.GenerativeModel(
  model_name="gemini-2.0-flash-exp",
  generation_config=generation_config,
)

chat_session = model.start_chat(
  history=[
    {
      "role": "user",
      "parts": [
        "I want to prompt the chatbot using data like this: \n**Average Rating:** 2.0\n**Sentiment:** negative: 100.0%, positive: 0.0%\n**Key Emotions:** frustration: 100.0%\n**Feedback by Topic:**\n- Exam: Main concerns are about Time Constraint, Difficulty of the Exam.\n- Assignment: Main concerns are about Resources of the Assignment.\n- Lecturer: Main concerns are about Teaching Style of the Lecturer.\n- Grading: Main concerns are about Fairness of the Grading.\n**Time-Based Trend:** 2025-01 saw the lowest ratings. To ask if for recommendations to improve a module if the feedback is bad, or give praise. for example, feedback like, to address the student concerns about time constraints, you may want to allow more time for the exam or reduce the number of questions",
      ],
    },
    {
      "role": "model",
      "parts": [
        "Okay, I understand. You want me to use this structured feedback data to generate recommendations (if the feedback is bad) or praise (if the feedback is good) aimed at improving a module or reinforcing positive aspects. \n\nHere's how I can process that data and what kind of prompts I'll generate based on the provided information:\n\n**General Approach:**\n\n1.  **Identify Overall Sentiment:** The first step is to look at the `Average Rating` and `Sentiment` to determine the overall tone of the feedback. In your example, it's clearly negative due to the low rating and 100% negative sentiment.\n2.  **Extract Key Concerns:**  I will then extract the main concerns from the `Key Emotions` and `Feedback by Topic` sections.\n3.  **Time Trend Analysis:** Use `Time-Based Trend` to add more context and see if the issues are recent or persistent.\n4.  **Generate Specific Recommendations/Praise:** I will formulate actionable suggestions tailored to the identified issues, and if the feedback were positive, I would provide praise related to the specific strengths.\n\n**Example Based on YOUR Provided Data:**\n\nHere's how I would process the data you gave me and what prompt I would generate:\n\n**Analysis:**\n\n*   **Overall:** Negative feedback (2.0 average rating, 100% negative sentiment).\n*   **Key Emotions:** High frustration.\n*   **Specific Concerns:**\n    *   **Exam:** Time constraints and difficulty.\n    *   **Assignment:** Resources inadequacy.\n    *   **Lecturer:** Teaching style.\n    *   **Grading:** Fairness.\n*   **Time Trend:** Issues were worst in 2025-01.\n\n**Generated Prompt:**\n\n\"The module received overwhelmingly negative feedback, with an average rating of 2.0 and 100% negative sentiment. Students express high levels of frustration. Specifically, they have significant concerns about:\n\n*   **Exam:** The exam is perceived as too time-constrained and too difficult. To address this, consider allowing more time for the exam or reducing the number of questions. You may also want to review the difficulty level to ensure it is appropriate for the learning outcomes.\n*   **Assignment:** Students are dissatisfied with the resources provided for the assignments. Improving the clarity, accessibility and relevance of the resources is critical.\n*  **Lecturer:**  Students have issues with the lecturer's teaching style. To address this, please review and enhance the teaching approach with more active, engaging methods.\n*   **Grading:** Students believe the grading is not fair.  You should re-evaluate the grading criteria and ensure they are transparent, consistent, and clearly communicated to students.\n\nThe issues were particularly pronounced in 2025-01. It is imperative to address these concerns urgently to enhance student satisfaction and learning outcomes in this module.\"\n\n**How the Prompt Adapts:**\n\n*   **Positive Feedback:** If the average rating and sentiment were positive, the prompt would shift to highlighting the positive points, for example:\n    *   \"This module received positive feedback with an average rating of 4.5 and 90% positive sentiment. Students are pleased with the...\" \n    *   \"Students praise the lecturer's engagement and accessibility. Please continue the excellent work\"\n*   **Mixed Feedback:** If there was a mix of positive and negative feedback, I would generate prompts that acknowledges both and highlights areas for improvement.\n    *   \"The module received mixed feedback, with an average rating of 3.5. While students expressed satisfaction with the teaching style of the lecturer, there are concerns about the resources provided for assignments.\"\n\n**Key Improvements of the Prompt:**\n\n*   **Actionable Recommendations:**  The prompt doesn't just state problems; it gives concrete, actionable suggestions for improvement.\n*   **Specific and Tailored:** Recommendations are tied directly to specific feedback topics.\n*   **Contextualized:** Time-based trends add context to the issues.\n*   **Urgency (if needed):** If feedback is severely negative, it conveys a sense of urgency.\n\n**How to Use This:**\n\n1.  **Feed the Data:** Give me the structured data like you provided.\n2.  **Receive a Prompt:** I will generate a tailored prompt based on the analysis.\n3.  **Use the Prompt:** Use my prompt as a basis for developing reports, informing instructors, or designing improvements to your educational module.\n\nLet me know if you have any other scenarios you'd like to explore or if you want to refine this approach further! I'm ready to help.\n",
      ],
    },
    {
      "role": "user",
      "parts": [
        "based on this, I would prompt the chatbot like  for example, I am a teacher of a module Programming, and this is the summary of my feedback from my students: Overall: Negative feedback (2.0 average rating, 100% negative sentiment).\n\nKey Emotions: High frustration.\n\nSpecific Concerns:\n\nExam: Time constraints and difficulty.\n\nAssignment: Resources inadequacy.\n\nLecturer: Teaching style.\n\nGrading: Fairness.\n\nTime Trend: Issues were worst in 2025-01. How can I address each concern (i.e,) improve and prevent it to improve my feedback and how do I address the emotions and sentiment",
      ],
    },
    {
      "role": "model",
      "parts": [
        "Okay, I understand. You want to use the summarized feedback to prompt the chatbot to generate actionable strategies for you, as the teacher of the \"Programming\" module, to address the specific concerns, improve future feedback, and manage the negative emotions and sentiment.\n\nHere's how I'd approach prompting the chatbot, along with explanations of why I'm including each element:\n\n**Prompt:**\n\n\"I am a teacher for the 'Programming' module. I recently received student feedback, which indicates significant areas for improvement. Here's a summary of the feedback:\n\n**Overall:** Negative feedback (2.0 average rating, 100% negative sentiment).\n\n**Key Emotions:** High frustration.\n\n**Specific Concerns:**\n\n*   **Exam:** Time constraints and difficulty.\n*   **Assignment:** Resources inadequacy.\n*   **Lecturer:** Teaching style.\n*   **Grading:** Fairness.\n\n**Time Trend:** Issues were worst in 2025-01.\n\nGiven this feedback:\n\n1.  **Actionable Strategies:** Please provide specific, actionable strategies I can implement to address each of the identified concerns (exam, assignment, lecturer, and grading) to improve student learning outcomes and future feedback.\n2.  **Prevention:** How can I proactively address and prevent these issues from recurring in future iterations of the module? Please propose specific steps and methods.\n3.  **Emotional Management:** How can I address the high levels of frustration students are expressing, and improve the overall negative sentiment toward the module? What specific communication or support strategies should I consider?\n4. **Prioritisation:** Based on the above, which areas should I focus on first, and what is the rational for the suggested priortisation?\n\nI'm looking for practical, concrete advice that I can immediately start implementing. Please provide a comprehensive and structured response covering these points.\"\n\n**Explanation of the Prompt Components:**\n\n1.  **Context and Role:**\n    *   \"I am a teacher for the 'Programming' module.\" - This sets the context and tells the chatbot your role and which specific module the feedback is about. This is crucial to get tailored advice.\n2.  **Clear Summary of Feedback:**\n    *   The summary you provided is clearly presented and ensures the chatbot has a good understanding of the problem.\n3.  **Specific Requests:**\n    *   **Actionable Strategies:** Asking for specific, actionable strategies gets you past generic advice and into concrete steps.\n    *   **Prevention:** This forces the chatbot to think proactively and beyond just fixing the current problems.\n    *   **Emotional Management:** This tackles the emotional impact of the poor feedback, which is critical for student well-being and future engagement.\n    *   **Prioritisation:** Helps you to decide where to focus your efforts, given limited time and resources.\n4.  **Emphasis on Practicality:**\n    *   \"I'm looking for practical, concrete advice that I can immediately start implementing.\" - This reinforces that you need real-world advice, not abstract concepts.\n5.  **Request for Structure:**\n    *   \"Please provide a comprehensive and structured response covering these points.\" - This encourages the chatbot to organize its answer for clarity.\n\n**Expected Chatbot Response:**\n\nThe chatbot should now provide you with detailed suggestions that might include:\n\n*   **Exam:**\n    *   **Time Management:** Strategies for better exam time allocation, perhaps practice exams or providing time recommendations.\n    *   **Difficulty:** Suggestions for revising exam questions and how to test material appropriately.\n*   **Assignment:**\n    *   **Resource Improvement:** How to create or find better resources, and how to make them available and understandable.\n    *   **Assignment Clarification:** How to make the assignment goals clearer and more defined.\n*   **Lecturer:**\n    *   **Teaching Style:** Strategies for improving engagement, using different methods, using formative assessments, making more time for individual questions.\n    *   **Communication:** How to have more effective office hours, and respond to queries in a timely manner.\n*   **Grading:**\n    *   **Clarity:** Making grading criteria more transparent and consistent.\n    *   **Fairness:** How to eliminate any perceived bias, ensuring each student is graded fairly.\n*   **Prevention:** Suggestions such as regular surveys, gathering of feedback and changes to the learning methodology.\n*   **Emotional Management:** Open lines of communication, encourage feedback, empathy and acknowledging student struggles.\n*   **Prioritization:** Recommendations on which areas should take precedence and how to prioritize them.\n\n**Why This Prompt Is Effective:**\n\n*   **Comprehensive:** It addresses the problem from all angles.\n*   **Action-Oriented:** It asks for specific, doable steps.\n*   **Empathy Aware:** It addresses both the practical and emotional aspects of the situation.\n*   **Future Focused:** It seeks ways to prevent the problems from recurring.\n\nBy using this prompt, you'll likely receive a comprehensive guide for improving your \"Programming\" module, student satisfaction, and your own teaching approach. Remember to follow up on the initial chatbot response, ask clarifying questions and use this as the start of the solution.\n",
      ],
    },
    {
      "role": "user",
      "parts": [
        "An example of feedback from the chatbot might be \"overall, the feedback on your module is {sentiment}, with students feeling {emotion}. This is mainly because students are concerned about the time constraints and difficulty of the exam, the lack of resources available for Assignments and the teaching style of the lecturer. To address the concerns about exams, consider reducing the number of questions in the paper or extending the time ... something similar for each topic.\" Remember that it is a recommendation/suggestion, don't make it seem like you are an expert and be mindful of the teacher's feelings",
      ],
    },
    {
      "role": "model",
      "parts": [
        "Okay, I understand. You want the chatbot to generate feedback recommendations that are:\n\n*   **Non-authoritative:** Not coming across as an expert, but rather as a helpful assistant offering suggestions.\n*   **Empathetic:** Mindful of the teacher's feelings, especially considering the negative feedback received.\n*   **Specific:** Providing concrete examples, just as mentioned in your example about the exam.\n*   **Structured:** Addressing each topic (exam, assignment, lecturer, grading) individually.\n\nHere's how I'd frame a chatbot response that meets those criteria:\n\n**Example Chatbot Response:**\n\n\"Okay, let's look at this feedback for the 'Programming' module together. It sounds like you've got some areas where students are feeling frustrated, and that can be tough to hear.\n\nOverall, the feedback on your module is **largely negative**, with students feeling **high levels of frustration**. This seems mainly because students are concerned about:\n\n*   **Exam:** The time constraints and difficulty of the exam are a major point of concern. To address this, you might consider reducing the number of questions on the paper or extending the time allowed for the exam. This could help ease the pressure students feel. You could also review the exam questions to ensure they are aligned with the module learning outcomes and difficulty level.\n*   **Assignment:** Students have indicated there is a lack of resources available for assignments. Perhaps reviewing the current resources and seeing if additional material is needed, or if the available material can be made clearer would be helpful. You could consider adding supplemental material, pointers, or more detailed instructions to enhance student understanding.\n*   **Lecturer:** The teaching style of the lecturer is also a concern for students. It could be helpful to explore how the course is being taught to see if there are other teaching approaches that would engage students more effectively. Maybe try to incorporate some more active learning techniques, or look into other methods that could make the content more accessible.\n*   **Grading:** Finally, there are concerns about the fairness of the grading. Reviewing and clarifying the grading criteria with students, and providing rubrics of how grades will be awarded could be a step to consider. This could improve the level of trust and transparency of the grading.\n\nLooking at the time trend, it appears the issues were most pronounced in 2025-01, so this might be a good area to focus on.\n\nThese are just a few ideas, of course. I'm here to help you explore all the options and improve the experience for your students, and I am confident that by addressing these specific points, the situation can improve.\"\n\n**Key Features of This Response:**\n\n*   **Empathetic Opening:** Phrases like \"It sounds like you've got some areas where students are feeling frustrated, and that can be tough to hear\" acknowledges the emotional impact of negative feedback.\n*   **Non-Authoritative Language:** The use of phrases like \"you might consider,\" \"perhaps,\" \"it could be helpful,\" and \"maybe try to\" makes the suggestions feel less like dictates and more like helpful ideas. It takes away the \"expert\" tone and offers advice, not commands.\n*   **Specific Examples:** Each suggestion includes a concrete example of how the identified problem could be addressed. This includes actionable ideas like \"reducing the number of questions\" or \"providing rubrics\".\n*   **Structured Format:** Each concern (exam, assignment, lecturer, grading) is addressed individually for clarity.\n*   **Focus on Improvement:** The response focuses on helping the teacher improve, not dwelling on the negative feedback itself.\n*   **Positive Ending:** The response ends on a note of support and confidence in the teacher's ability to improve the situation.\n*   **Focus on the Data:** The inclusion of \"looking at the time trend, it appears the issues were most pronounced in 2025-01\" reinforces the idea that the suggestions are backed by the data, not arbitrary recommendations.\n\n**How to Use This:**\n\nWhen prompting the chatbot, you can include instructions like:\n\n\"Please generate a response that acknowledges that feedback can be difficult to receive. Your response should be non-authoritative, offering suggestions rather than commands, and mindful of the teacher's feelings. Please provide specific and actionable recommendations. Include a positive ending.\"\n\nBy providing these guidelines to the chatbot, you should receive a response that is more helpful and less likely to overwhelm the teacher with seemingly harsh or prescriptive advice. Remember to further question the chatbot to tailor the advice even further.\n",
      ],
    },
  ]
)

response = chat_session.send_message('''
I am a teacher of a module Programming, and this is the summary of my feedback from my students:
Overall: Negative feedback (2.0 average rating, 100% negative sentiment).

Key Emotions: High frustration.

Specific Concerns:

Exam: Time constraints and difficulty.

Assignment: Resources inadequacy.

Lecturer: Teaching style.

Grading: Fairness.

Time Trend: Issues were worst in 2025-01. How can I address each concern (i.e,) improve and prevent it to improve my feedback and how do I address the emotions and sentiment
''')

print(response.text)

# How are recommedations for improvement given to teachers?
Note: A rule-based approach was considered but it would have been tedious to build and perhaps inaccurate as it would be difficult to cover all the cases of feedback to come up with a response that has the potential to actually benefit teachers.

**Approach**

* An LLM-based approach (google gemini) has been used where the main challenges are tuning the parameters of the model and building a really good prompt to provide recommendations that are tailored and actually useful.
* The prompt is built using a summary of the data. Specifically:

  1.   Overall Feedback (i.e., mostly positive or negative)
  2.   Main Emotions (e.g., frustration, joy)
  3.   Specific Concerns relating to each topic (e.g., Exam difficulty). Initially, I was going to use topics only, but the recommendation was not specific enough. E.g., Most of the reviews about exams being negative does not specifically pinpoint what the teacher can improve, despite giving an indication. Therefore, I decided to apply subtopics for each topic label that will be used to extract the specific concerns. One topic may have mulitple concerns.
  4. Time trend (when emotions and sentiment were worst/best)
  5. Asking for specific, emphathetic (teachers are human so would not want recommendations to be rude) and actionable recommendations to address the feedback.

expansion? teachers may be able to like/dislike feedback which can be used to tune the model parameters and prompt.



In [ ]:
import os
import google.generativeai as genai


genai.configure(api_key='SECRET')

# Create the model
generation_config = {
  "temperature": 0.7, # need to check temp param
  "top_p": 0.95,
  "top_k": 40,
  "max_output_tokens": 8192,
  "response_mime_type": "text/plain",
}

model = genai.GenerativeModel(
  model_name="gemini-2.0-flash-exp",
  generation_config=generation_config,
)

chat_session = model.start_chat(
  history=[
    {
      "role": "user",
      "parts": [
        "something to give recommendations to teachers to improve modules based on student feedback on modules that they teach\n",
      ],
    },
    {
      "role": "model",
      "parts": [
        "Okay, let's craft something that can provide teachers with actionable recommendations based on student feedback on their modules. Here's a breakdown of how it could work, along with examples:\n\n**Core Components:**\n\n1. **Feedback Collection:**\n   * **Structured Surveys:** This is the most common method. Questions could cover:\n      * **Clarity of Content:** (e.g., Was the material presented in a clear and understandable way?)\n      * **Engagement:** (e.g., Did you find the module engaging? Did it hold your interest?)\n      * **Organization:** (e.g., Was the module well-organized? Did the flow of topics make sense?)\n      * **Usefulness:** (e.g., How relevant was the material to your learning goals?)\n      * **Pacing:** (e.g., Was the pace of the module appropriate?)\n      * **Assessment:** (e.g., Were the assessments fair and aligned with the content?)\n      * **Instructor's Role:** (e.g., Was the instructor responsive and helpful?)\n      * **Overall Experience:** (e.g., How satisfied were you with the module overall?)\n   * **Open-Ended Feedback:** Allow students to provide written comments on what they liked, what they didn't, and suggestions for improvement.\n   * **Anonymous Feedback:** Encourage honest responses by ensuring anonymity.\n\n2. **Data Analysis:**\n   * **Quantitative Analysis:**\n      * **Average Scores:** Calculate average scores for each question to identify areas of strength and weakness.\n      * **Frequency Distributions:** Analyze how often students selected specific responses (e.g., how many students rated a question as \"Excellent\").\n      * **Trend Analysis:** Track changes in scores over time to monitor improvement or decline in different areas.\n   * **Qualitative Analysis:**\n      * **Theme Identification:** Look for common themes and patterns in the open-ended feedback.\n      * **Sentiment Analysis:** Use tools to analyze the overall tone (positive, negative, neutral) of the comments.\n\n3. **Recommendation Generation:** This is the key part. Here's how to generate meaningful recommendations:\n   * **Based on Quantitative Data:**\n      * **Low-Scoring Areas:** If a question consistently scores low, suggest specific actions (e.g., \"The question 'Clarity of Content' scored below average. Consider providing more detailed examples or explanations.\")\n      * **Inconsistent Ratings:** If there's a wide spread of ratings, this might indicate that the content appeals to some but not others. Recommend offering optional extensions or different approaches to the material.\n   * **Based on Qualitative Data:**\n       * **Recurring Themes:** If multiple students mention that the pacing is too fast, recommend slowing down. If multiple students mention that the instructor's responsiveness is good, highlight this strength.\n       * **Specific Suggestions:** If a student suggests adding more real-world examples, suggest incorporating them.\n   * **Focus on Actionable Advice:** Recommendations should be specific and easily translated into concrete changes. Avoid vague statements.\n   * **Provide Context:** Present data in a way that is clear and understandable for the teacher, along with comparisons to previous years or other similar modules (if available).\n   * **Highlight Strengths:**  Don't focus solely on weaknesses.  Also include positive feedback to reinforce what is working well.\n\n**Example Scenario and Recommendations:**\n\n**Scenario:**\n\n* **Module:** \"Introduction to Programming\"\n* **Feedback:**\n   * **Quantitative:**  Low average score on the \"Clarity of Content\" question.\n   * **Qualitative:**  Multiple students commented that \"the concepts were confusing\" and \"the examples were too abstract.\"\n\n**Generated Recommendations:**\n\n* **Recommendation 1 (Quantitative):**  \"The average score for 'Clarity of Content' was 3.2 out of 5, which is below the average for similar introductory modules. Consider reviewing the way key programming concepts are presented. Make use of visual aids, analogies, and step-by-step explanations to enhance clarity.\"\n* **Recommendation 2 (Qualitative):** \"Multiple students commented that the concepts were confusing and the examples were too abstract.  Consider incorporating more real-world examples that are directly relatable to student's experiences. For instance, instead of using abstract variables, try a grocery shopping analogy.\"\n* **Recommendation 3 (Qualitative):** \"Students also suggested breaking down the code examples into smaller steps. For the next iteration, try creating code walkthroughs where you explain what each line does and why.\"\n* **Positive Reinforcement:** \"Students consistently praised your enthusiasm and passion for the subject.\"\n* **Next Steps:** \"We recommend reviewing the module materials and incorporating these suggestions for the next iteration. A follow-up session in a few weeks can further help you address any concerns.\"\n\n**Key Features of a Good Recommendation System:**\n\n* **User-Friendly Interface:** The tool should be easy for teachers to use and understand, even if they are not data experts.\n* **Customizable:** Allow teachers to focus on specific aspects of their teaching.\n* **Regular Updates:** The tool should provide updated data on a regular basis, allowing teachers to track their progress.\n* **Support and Resources:** Provide resources and training to help teachers understand the recommendations and implement changes.\n\n**Technology to Use:**\n\n* **Survey Platforms:** (e.g., Google Forms, SurveyMonkey, Typeform)\n* **Data Analysis Tools:** (e.g., Excel, Google Sheets, Python with Pandas, R)\n* **Visualization Tools:** (e.g., Tableau, Power BI)\n* **Natural Language Processing (NLP):**  For more advanced qualitative analysis\n\nBy implementing a comprehensive system for collecting, analyzing, and generating recommendations based on student feedback, teachers can gain valuable insights into the effectiveness of their modules, leading to a more engaging and effective learning experience for students. Remember that this is an iterative process. Regularly collect feedback and refine your approach to continually improve your teaching.\n",
      ],
    },
  ]
)

response = chat_session.send_message('''
I'm a teacher of a 'Programming' module and I recently received feedback with the following summary:

**Overall:** Negative feedback (2.0 average rating, 100% negative sentiment).

**Key Emotions:** High frustration.

**Specific Concerns:**

*   **Exam:** Time constraints and difficulty.
*   **Assignment:** Resources inadequacy.
*   **Lecturer:** Teaching style.
*   **Grading:** Fairness.

**Time Trend:** Issues were worst in 2025-01.

I need guidance on how to address this feedback effectively. Specifically, I'd like recommendations that are:

1.  **Specific & Actionable:** For each area of concern (Exam, Assignment, Lecturer, and Grading), provide me with practical, specific actions I can take to address the issues.
2.  **Preventative:** How can I prevent these same issues from occurring again in future semesters or iterations of the module? What methods can I use for ongoing feedback and improvement?
3.  **Emotionally Sensitive:** How can I best acknowledge and address the high levels of frustration and negative sentiment that students are expressing? What communication strategies should I use to rebuild trust and create a more positive learning environment?
4.  **Prioritised:** If there are multiple things that I could do, which should I focus on first and why?
5.  **Considered for Time Trend:** How should I interpret the time trend to help in resolving the issues.

I'm looking for a response that is practical, empathetic, and specific to my situation as a teacher for a 'Programming' module.
''')
print(response.text)

Okay, let's break down this feedback and create a plan of action. This is a challenging situation, but with a focused approach, you can turn things around.

**Understanding the Severity:**

*   **2.0 Average Rating & 100% Negative Sentiment:** This is a clear indication of significant problems. It's critical to take this seriously and acknowledge the students' dissatisfaction.
*   **High Frustration:** This isn't just about academic performance; students are experiencing strong negative emotions. Addressing this is as important as addressing the technical issues.
*   **Time Trend (Worst in 2025-01):** This suggests a specific point in time where things may have gone particularly wrong. This is crucial for pinpointing what changes might have negatively impacted the module at that time.

**Action Plan:**

**1. Specific & Actionable Recommendations:**

*   **Exam:**
    *   **Problem:** Time constraints and difficulty.
    *   **Actions:**
        *   **Review Exam Content:** Analyze the 

# Testing for a kind of positive review

In [ ]:
import os
import google.generativeai as genai


genai.configure(api_key='SECRET')

# Create the model
generation_config = {
  "temperature": 0.7, # need to check temp param
  "top_p": 0.95,
  "top_k": 40,
  "max_output_tokens": 8192,
  "response_mime_type": "text/plain",
}

model = genai.GenerativeModel(
  model_name="gemini-2.0-flash-exp",
  generation_config=generation_config,
)

chat_session = model.start_chat(
  history=[
    {
      "role": "user",
      "parts": [
        "something to give recommendations to teachers to improve modules based on student feedback on modules that they teach\n",
      ],
    },
    {
      "role": "model",
      "parts": [
        "Okay, let's craft something that can provide teachers with actionable recommendations based on student feedback on their modules. Here's a breakdown of how it could work, along with examples:\n\n**Core Components:**\n\n1. **Feedback Collection:**\n   * **Structured Surveys:** This is the most common method. Questions could cover:\n      * **Clarity of Content:** (e.g., Was the material presented in a clear and understandable way?)\n      * **Engagement:** (e.g., Did you find the module engaging? Did it hold your interest?)\n      * **Organization:** (e.g., Was the module well-organized? Did the flow of topics make sense?)\n      * **Usefulness:** (e.g., How relevant was the material to your learning goals?)\n      * **Pacing:** (e.g., Was the pace of the module appropriate?)\n      * **Assessment:** (e.g., Were the assessments fair and aligned with the content?)\n      * **Instructor's Role:** (e.g., Was the instructor responsive and helpful?)\n      * **Overall Experience:** (e.g., How satisfied were you with the module overall?)\n   * **Open-Ended Feedback:** Allow students to provide written comments on what they liked, what they didn't, and suggestions for improvement.\n   * **Anonymous Feedback:** Encourage honest responses by ensuring anonymity.\n\n2. **Data Analysis:**\n   * **Quantitative Analysis:**\n      * **Average Scores:** Calculate average scores for each question to identify areas of strength and weakness.\n      * **Frequency Distributions:** Analyze how often students selected specific responses (e.g., how many students rated a question as \"Excellent\").\n      * **Trend Analysis:** Track changes in scores over time to monitor improvement or decline in different areas.\n   * **Qualitative Analysis:**\n      * **Theme Identification:** Look for common themes and patterns in the open-ended feedback.\n      * **Sentiment Analysis:** Use tools to analyze the overall tone (positive, negative, neutral) of the comments.\n\n3. **Recommendation Generation:** This is the key part. Here's how to generate meaningful recommendations:\n   * **Based on Quantitative Data:**\n      * **Low-Scoring Areas:** If a question consistently scores low, suggest specific actions (e.g., \"The question 'Clarity of Content' scored below average. Consider providing more detailed examples or explanations.\")\n      * **Inconsistent Ratings:** If there's a wide spread of ratings, this might indicate that the content appeals to some but not others. Recommend offering optional extensions or different approaches to the material.\n   * **Based on Qualitative Data:**\n       * **Recurring Themes:** If multiple students mention that the pacing is too fast, recommend slowing down. If multiple students mention that the instructor's responsiveness is good, highlight this strength.\n       * **Specific Suggestions:** If a student suggests adding more real-world examples, suggest incorporating them.\n   * **Focus on Actionable Advice:** Recommendations should be specific and easily translated into concrete changes. Avoid vague statements.\n   * **Provide Context:** Present data in a way that is clear and understandable for the teacher, along with comparisons to previous years or other similar modules (if available).\n   * **Highlight Strengths:**  Don't focus solely on weaknesses.  Also include positive feedback to reinforce what is working well.\n\n**Example Scenario and Recommendations:**\n\n**Scenario:**\n\n* **Module:** \"Introduction to Programming\"\n* **Feedback:**\n   * **Quantitative:**  Low average score on the \"Clarity of Content\" question.\n   * **Qualitative:**  Multiple students commented that \"the concepts were confusing\" and \"the examples were too abstract.\"\n\n**Generated Recommendations:**\n\n* **Recommendation 1 (Quantitative):**  \"The average score for 'Clarity of Content' was 3.2 out of 5, which is below the average for similar introductory modules. Consider reviewing the way key programming concepts are presented. Make use of visual aids, analogies, and step-by-step explanations to enhance clarity.\"\n* **Recommendation 2 (Qualitative):** \"Multiple students commented that the concepts were confusing and the examples were too abstract.  Consider incorporating more real-world examples that are directly relatable to student's experiences. For instance, instead of using abstract variables, try a grocery shopping analogy.\"\n* **Recommendation 3 (Qualitative):** \"Students also suggested breaking down the code examples into smaller steps. For the next iteration, try creating code walkthroughs where you explain what each line does and why.\"\n* **Positive Reinforcement:** \"Students consistently praised your enthusiasm and passion for the subject.\"\n* **Next Steps:** \"We recommend reviewing the module materials and incorporating these suggestions for the next iteration. A follow-up session in a few weeks can further help you address any concerns.\"\n\n**Key Features of a Good Recommendation System:**\n\n* **User-Friendly Interface:** The tool should be easy for teachers to use and understand, even if they are not data experts.\n* **Customizable:** Allow teachers to focus on specific aspects of their teaching.\n* **Regular Updates:** The tool should provide updated data on a regular basis, allowing teachers to track their progress.\n* **Support and Resources:** Provide resources and training to help teachers understand the recommendations and implement changes.\n\n**Technology to Use:**\n\n* **Survey Platforms:** (e.g., Google Forms, SurveyMonkey, Typeform)\n* **Data Analysis Tools:** (e.g., Excel, Google Sheets, Python with Pandas, R)\n* **Visualization Tools:** (e.g., Tableau, Power BI)\n* **Natural Language Processing (NLP):**  For more advanced qualitative analysis\n\nBy implementing a comprehensive system for collecting, analyzing, and generating recommendations based on student feedback, teachers can gain valuable insights into the effectiveness of their modules, leading to a more engaging and effective learning experience for students. Remember that this is an iterative process. Regularly collect feedback and refine your approach to continually improve your teaching.\n",
      ],
    },
  ]
)

response = chat_session.send_message('''
I’m a teacher of a ‘Programming’ module, and I recently received feedback with the following summary:

Overall: Positive feedback (4.5 average rating, 70% positive sentiment).

Key Emotions: High joy, excitement, and admiration.

Main Areas of Commentary:

Exam: Clear and fair.
Assignment: Well-resourced and challenging.
Lecturer: Engaging and informative teaching style.
Grading: Fair and transparent.
Time Trend: Positive feedback consistently increased throughout the semester, especially after midterm.

I need guidance on how to build on this positive feedback and further enhance the student experience. Specifically, I'd like recommendations that are:

Specific & Actionable: For each area of positive commentary (Exam, Assignment, Lecturer, and Grading), provide me with practical, specific actions I can take to maintain or improve these areas.
Preventative: How can I prevent complacency and continue improving these areas in future semesters or iterations of the module?
Emotionally Sensitive: How can I acknowledge the positive feedback and use it to motivate students further, while maintaining a positive learning environment?
Prioritised: If there are multiple things I could do to improve, which areas should I focus on first and why?
Considered for Time Trend: How can I use the positive time trend to ensure that future semesters continue on this upward trajectory?
I’m looking for a response that is practical, forward-thinking, and specific to my situation as a teacher for a ‘Programming’ module.
''')
print(response.text)

Okay, let's break down this positive feedback and create a plan to build upon it, specifically tailored for your "Programming" module.

**Overall Strategy:** Your feedback is fantastic! The key here isn't to overhaul everything, but to refine what's already working well and prevent any potential backsliding. We’ll focus on specific, actionable steps and strategically prioritize.

**1. Actionable Recommendations by Area:**

*   **Exam (Clear and Fair):**
    *   **Maintain Clarity:**
        *   **Action:** Continue using the same clear and concise language in exam questions and instructions.
        *   **Action:** Ensure exam questions directly align with topics and examples covered in lectures and assignments.
        *   **Action:** Consider providing a sample exam or practice questions with solutions before the actual exam.
    *   **Enhance Fairness:**
        *   **Action:** Review past exams, ensuring there’s a good balance of difficulty and topic coverage.
        *   **Action:

# Testing a mixture of data



In [ ]:
import os
import google.generativeai as genai


genai.configure(api_key='SECRET')

# Create the model
generation_config = {
  "temperature": 0.7, # need to check temp param
  "top_p": 0.95,
  "top_k": 40,
  "max_output_tokens": 8192,
  "response_mime_type": "text/plain",
}

model = genai.GenerativeModel(
  model_name="gemini-2.0-flash-exp",
  generation_config=generation_config,
)

chat_session = model.start_chat(
  history=[
    {
      "role": "user",
      "parts": [
        "something to give recommendations to teachers to improve modules based on student feedback on modules that they teach\n",
      ],
    },
    {
      "role": "model",
      "parts": [
        "Okay, let's craft something that can provide teachers with actionable recommendations based on student feedback on their modules. Here's a breakdown of how it could work, along with examples:\n\n**Core Components:**\n\n1. **Feedback Collection:**\n   * **Structured Surveys:** This is the most common method. Questions could cover:\n      * **Clarity of Content:** (e.g., Was the material presented in a clear and understandable way?)\n      * **Engagement:** (e.g., Did you find the module engaging? Did it hold your interest?)\n      * **Organization:** (e.g., Was the module well-organized? Did the flow of topics make sense?)\n      * **Usefulness:** (e.g., How relevant was the material to your learning goals?)\n      * **Pacing:** (e.g., Was the pace of the module appropriate?)\n      * **Assessment:** (e.g., Were the assessments fair and aligned with the content?)\n      * **Instructor's Role:** (e.g., Was the instructor responsive and helpful?)\n      * **Overall Experience:** (e.g., How satisfied were you with the module overall?)\n   * **Open-Ended Feedback:** Allow students to provide written comments on what they liked, what they didn't, and suggestions for improvement.\n   * **Anonymous Feedback:** Encourage honest responses by ensuring anonymity.\n\n2. **Data Analysis:**\n   * **Quantitative Analysis:**\n      * **Average Scores:** Calculate average scores for each question to identify areas of strength and weakness.\n      * **Frequency Distributions:** Analyze how often students selected specific responses (e.g., how many students rated a question as \"Excellent\").\n      * **Trend Analysis:** Track changes in scores over time to monitor improvement or decline in different areas.\n   * **Qualitative Analysis:**\n      * **Theme Identification:** Look for common themes and patterns in the open-ended feedback.\n      * **Sentiment Analysis:** Use tools to analyze the overall tone (positive, negative, neutral) of the comments.\n\n3. **Recommendation Generation:** This is the key part. Here's how to generate meaningful recommendations:\n   * **Based on Quantitative Data:**\n      * **Low-Scoring Areas:** If a question consistently scores low, suggest specific actions (e.g., \"The question 'Clarity of Content' scored below average. Consider providing more detailed examples or explanations.\")\n      * **Inconsistent Ratings:** If there's a wide spread of ratings, this might indicate that the content appeals to some but not others. Recommend offering optional extensions or different approaches to the material.\n   * **Based on Qualitative Data:**\n       * **Recurring Themes:** If multiple students mention that the pacing is too fast, recommend slowing down. If multiple students mention that the instructor's responsiveness is good, highlight this strength.\n       * **Specific Suggestions:** If a student suggests adding more real-world examples, suggest incorporating them.\n   * **Focus on Actionable Advice:** Recommendations should be specific and easily translated into concrete changes. Avoid vague statements.\n   * **Provide Context:** Present data in a way that is clear and understandable for the teacher, along with comparisons to previous years or other similar modules (if available).\n   * **Highlight Strengths:**  Don't focus solely on weaknesses.  Also include positive feedback to reinforce what is working well.\n\n**Example Scenario and Recommendations:**\n\n**Scenario:**\n\n* **Module:** \"Introduction to Programming\"\n* **Feedback:**\n   * **Quantitative:**  Low average score on the \"Clarity of Content\" question.\n   * **Qualitative:**  Multiple students commented that \"the concepts were confusing\" and \"the examples were too abstract.\"\n\n**Generated Recommendations:**\n\n* **Recommendation 1 (Quantitative):**  \"The average score for 'Clarity of Content' was 3.2 out of 5, which is below the average for similar introductory modules. Consider reviewing the way key programming concepts are presented. Make use of visual aids, analogies, and step-by-step explanations to enhance clarity.\"\n* **Recommendation 2 (Qualitative):** \"Multiple students commented that the concepts were confusing and the examples were too abstract.  Consider incorporating more real-world examples that are directly relatable to student's experiences. For instance, instead of using abstract variables, try a grocery shopping analogy.\"\n* **Recommendation 3 (Qualitative):** \"Students also suggested breaking down the code examples into smaller steps. For the next iteration, try creating code walkthroughs where you explain what each line does and why.\"\n* **Positive Reinforcement:** \"Students consistently praised your enthusiasm and passion for the subject.\"\n* **Next Steps:** \"We recommend reviewing the module materials and incorporating these suggestions for the next iteration. A follow-up session in a few weeks can further help you address any concerns.\"\n\n**Key Features of a Good Recommendation System:**\n\n* **User-Friendly Interface:** The tool should be easy for teachers to use and understand, even if they are not data experts.\n* **Customizable:** Allow teachers to focus on specific aspects of their teaching.\n* **Regular Updates:** The tool should provide updated data on a regular basis, allowing teachers to track their progress.\n* **Support and Resources:** Provide resources and training to help teachers understand the recommendations and implement changes.\n\n**Technology to Use:**\n\n* **Survey Platforms:** (e.g., Google Forms, SurveyMonkey, Typeform)\n* **Data Analysis Tools:** (e.g., Excel, Google Sheets, Python with Pandas, R)\n* **Visualization Tools:** (e.g., Tableau, Power BI)\n* **Natural Language Processing (NLP):**  For more advanced qualitative analysis\n\nBy implementing a comprehensive system for collecting, analyzing, and generating recommendations based on student feedback, teachers can gain valuable insights into the effectiveness of their modules, leading to a more engaging and effective learning experience for students. Remember that this is an iterative process. Regularly collect feedback and refine your approach to continually improve your teaching.\n",
      ],
    },
  ]
)

response = chat_session.send_message('''
I’m a teacher of a ‘Programming’ module, and I recently received feedback with the following summary:

Overall: Mixed feedback (3.4 average rating, 55% positive sentiment, 45% negative sentiment).

Key Emotions: Mixed emotions: Frustration (40%), Joy (30%), Admiration (20%), Surprise (10%).

Specific Feedback for Each Topic:

Exam: Completely positive feedback – students appreciated the clarity and fairness.
Assignment: Mixed feedback – some found the resources inadequate, others enjoyed the challenge and found the materials useful.
Lecturer: Negative feedback – students felt the teaching style was unclear and difficult to follow.
Grading: Mixed feedback – some students were concerned about fairness, while others appreciated the transparency.
Time Trend: Positive feedback started increasing after midterm, but negative sentiment peaked around the midterm exams.

I need guidance on how to address the mixed feedback effectively for each topic. Specifically, I'd like recommendations that are:

Specific & Actionable: For each topic, provide practical, specific actions I can take based on whether the feedback is positive, negative, or mixed. For topics with positive feedback (e.g., Exam), how can I maintain or improve what is working? For topics with negative feedback (e.g., Lecturer), what actions can I take to address the concerns? For mixed topics (e.g., Assignment, Grading), how can I balance the positives and improve the negatives?
Preventative: How can I prevent the negative feedback from recurring in future semesters for the areas with negative or mixed feedback (e.g., Lecturer and Assignment)? How can I continue to reinforce the positive feedback in topics like Exam?
Emotionally Sensitive: How can I address the frustration and negative sentiment in the areas where students are dissatisfied (Lecturer, Grading), while reinforcing the positive emotions in the areas of praise (Exam)? What communication strategies can I use to show students I’ve heard their concerns and am acting on them?
Prioritised: Which topics should I focus on first and why? Should I prioritize improving the areas with negative feedback (e.g., Lecturer), or should I continue building on the strengths in areas with positive feedback (e.g., Exam)?
Considered for Time Trend: Given that positive feedback increased after midterm and negative sentiment peaked during the midterm exams, how should I interpret this trend to help resolve issues and continue improving? What steps can I take to ensure that positive trends continue?
I’m looking for a response that is practical, empathetic, and specific to my situation as a teacher for a ‘Programming’ module, taking into account the varying levels of feedback across different topics.
''')
print(response.text)

Okay, let's break down this feedback and create an action plan for your Programming module. Here's a response that is specific, actionable, preventative, emotionally sensitive, prioritised, and considers the time trend:

**Overall Summary:**

*   **Mixed Feedback:** 3.4 average rating, 55% positive sentiment, 45% negative sentiment.
*   **Emotions:** Frustration (40%), Joy (30%), Admiration (20%), Surprise (10%). This shows a strong emotional response to the module, indicating your teaching has impacted them, both positively and negatively.
*   **Time Trend:** Positive sentiment increased after midterm, negative peaked around midterm. This suggests the initial weeks or the midterm itself caused frustration.

**Prioritisation:**

You should **prioritize addressing the "Lecturer" feedback first.** This is because:
1.  **Fundamental Impact:** The instructor is central to the learning experience. If the teaching style is unclear, it impacts all other areas.
2.  **High Frustration:** The "L

In [2]:
pip install google-generativeai